In [8]:
import qutip
import numpy as np
from scipy.linalg import expm

First,  by inspection construct a target unitary for cyclic_3 swap.

$U|a, b, c\rangle = |b,c,a\rangle$.

\begin{equation}
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 1& 0& 0& 0\\
0& 1& 0& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 1& 0& 0\\
0& 0& 1& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 0& 1& 0\\
0& 0& 0& 1& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 0& 0& 1
\end{bmatrix}

\begin{bmatrix}
a\\
b\\
c\\
d\\
e\\
f\\
g\\
h
\end{bmatrix}
=
\begin{bmatrix}
a\\
e\\
b\\
f\\
c\\
g\\
d\\
h
\end{bmatrix}
\end{equation}

In [215]:
def build_circulator_U(phi_ab=0, phi_ac=0, phi_bc=np.pi/2, g_ab=1.0, g_ac=1.0, g_bc=1.0):
    #TODO: when sweeping phase parameters leave time-independent
    # when sweeping coupling parameters make time-dependet i.e gaussian shaped pusle
    #build raising and lowering operations
    a = qutip.operators.create(N=2)
    I2 = qutip.operators.identity(2)
    A = qutip.tensor(a, I2, I2)
    B = qutip.tensor(I2, a, I2)
    C = qutip.tensor(I2, I2, a)

    #construct circulator Hamiltonian
    H_ab = np.exp(1j * phi_ab)*A*B.dag() + np.exp(-1j * phi_ab)*A.dag()*B
    H_ac = np.exp(1j * phi_ac)*A*C.dag() + np.exp(-1j * phi_ac)*A.dag()*C
    H_bc = np.exp(1j * phi_bc)*B*C.dag() + np.exp(-1j * phi_bc)*B.dag()*C
    H = g_ab*H_ab + g_ac*H_ac + g_bc*H_bc
    
    #time evolution, if time dependent need to use master-equation
    #qutip.mesolve()
    # U = expm(1j*np.array(H))
    U = (1j*H).expm()
    return U

# #construct an example initial state
# inita = qutip.basis(2, 1)
# initb = qutip.basis(2, 0)
# initc = qutip.basis(2, 0)
# init = qutip.tensor(inita, initb, initc)
# #print(init)

# U = build_circulator_U()
# #apply unitary to initial state
# new_state = U*init
# print(new_state)


# #expected
# expected = qutip.tensor(initb, initc, inita)
# print(expected.overlap(new_state))

In [224]:
from scipy import optimize as opt
from itertools import product
#initial guess, here I'm letting phases also be parametric 
gk = [0,0,np.pi/2,1,1,1]

In [225]:
expected_U = qutip.Qobj(dims = [[2, 2, 2], [2, 2, 2]],inpt=(1.0+0j)* np.array([[1,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,1,0,0,0,0,0],[0,0,0,0,0,0,1,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1]]))
# gk = [1,1,1]
# template = build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])
# #1- np.abs(np.sum(np.matmul(template.full(), expected.conj().full())))/ (2*np.log2(expected.shape[0]))
# np.real(((expected_U.full() - template.full()) * (expected_U.full() - template.full()).conj()).mean() ** 0.5)
# print(expected_U)
# np.abs((expected_U*template.dag()).tr())/expected_U.shape[0]

In [234]:

#Option 1. Optimize inner product of quantum states
#Problem is that we want the expected inner product to be the same over all states not just 1 at a time
#construct an example initial state
def build_init_state(a,b,c):
    inita = qutip.basis(2, a)
    initb = qutip.basis(2, b)
    initc = qutip.basis(2, c)
    init = qutip.tensor(inita, initb, initc)
    return init

#expected
#expected_state = qutip.tensor(initb, initc, inita)
init = build_init_state(0,1,1)
expected_state = expected_U*init
# print(expected_state)


#can we improve this by iterating over all coherent states?
# def foo(gk):
#     sum = 0
#     k = 0
#     for a,b,c in product([0,1], repeat=3):
#         init = build_init_state(a,b,c)
#         inner_product = expected_state.overlap(build_circulator_U(phi_ab=gk[0], phi_ac=gk[1], phi_bc=gk[2],g_ab=gk[3], g_ac=gk[4], g_bc=gk[5])*init)
#         sum += 1 - np.real(np.conj(inner_product) * inner_product)
#         k+=1
#     return sum/k

#Option 2. Optmize target unitary difference
#Here I use the normalized overlap of the anstantz and target, the absolute value is taken to ignore global phase.
def foo(gk):
    template = build_circulator_U(phi_ab=gk[0], phi_ac=gk[1], phi_bc=gk[2],g_ab=gk[3], g_ac=gk[4], g_bc=gk[5])
    return 1- np.abs((expected_U*template.dag()).tr())/expected_U.shape[0]

opt_result = opt.minimize(fun=foo, x0=gk)
print(opt_result.fun)
print(opt_result.x)

# gk = opt_result.x
# U = build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])
# from pprint import pprint
# pprint(np.matrix(U))

0.5589351103432204
[ 5.23559410e-06 -5.23311737e-06 -3.57867373e-06  9.17717458e-01
  9.17717459e-01  9.17743734e-01]


This feels like evidence that for time-independent parameters, you just can't build the target unitary. You can get near-exact for states at a time, but if we iterate over just the most basic basis states we can't get a unitary that works for them all simultaneously. We should look into pulse control optimization methods supplied by Qutip, either GRAPE or CRAB which seem to be similar just with different optimization techniques.

https://docs.qdyn-library.net/MR-103/examples/CRAB/CRAB.html#Optimization

In [214]:
# def test_6_crab(self):
#     """
#     control.pulseoptim: Hadamard gate using CRAB algorithm
#     Apply guess and ramping pulse
#     assert that goal is achieved and fidelity error is below threshold
#     assert that starting amplitude is zero
#     """
#     # Hadamard
#     H_d = sigmaz()
#     H_c = [sigmax()]
#     U_0 = identity(2)
#     U_targ = hadamard_transform(1)

#     n_ts = 12
#     evo_time = 10
    
#     # Run the optimisation
#     result = cpo.opt_pulse_crab_unitary(H_d, H_c, U_0, U_targ, 
#             n_ts, evo_time, 
#             fid_err_targ=1e-5, 
#             alg_params={'crab_pulse_params':{'randomize_coeffs':False, 
#                                                 'randomize_freqs':False}},
#             init_coeff_scaling=0.5,
#             guess_pulse_type='GAUSSIAN', 
#             guess_pulse_params={'variance':0.1*evo_time},
#             guess_pulse_scaling=1.0, guess_pulse_offset=1.0,
#             amp_lbound=None, amp_ubound=None,
#             ramping_pulse_type='GAUSSIAN_EDGE', 
#             ramping_pulse_params={'decay_time':evo_time/100.0},
#             gen_stats=True)
#     assert_(result.goal_achieved, msg="Hadamard goal not achieved. "
#                 "Terminated due to: {}, with infidelity: {}".format(
#                 result.termination_reason, result.fid_err))
#     assert_almost_equal(result.fid_err, 0.0, decimal=3, 
#                         err_msg="Hadamard infidelity too high")
#     assert_almost_equal(result.final_amps[0, 0], 0.0, decimal=3, 
#                         err_msg="lead in amplitude not zero")
#     # Repeat with Qobj integration
#     result = cpo.opt_pulse_crab_unitary(H_d, H_c, U_0, U_targ, 
#             n_ts, evo_time, 
#             fid_err_targ=1e-5, 
#             alg_params={'crab_pulse_params':{'randomize_coeffs':False, 
#                                                 'randomize_freqs':False}},
#             dyn_params={'oper_dtype':Qobj},
#             init_coeff_scaling=0.5,
#             guess_pulse_type='GAUSSIAN', 
#             guess_pulse_params={'variance':0.1*evo_time},
#             guess_pulse_scaling=1.0, guess_pulse_offset=1.0,
#             amp_lbound=None, amp_ubound=None,
#             ramping_pulse_type='GAUSSIAN_EDGE', 
#             ramping_pulse_params={'decay_time':evo_time/100.0},
#             gen_stats=True)
#     assert_(result.goal_achieved, msg="Hadamard goal not achieved" 
#                                     "(Qobj integration). "
#                 "Terminated due to: {}, with infidelity: {}".format(
#                 result.termination_reason, result.fid_err))